In [ ]:
# main.py

from fastapi import FastAPI, Request, Form, UploadFile
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.orm import declarative_base, sessionmaker
from starlette.middleware.sessions import SessionMiddleware
from starlette.requests import Request
import csv

app = FastAPI()

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL)
metadata = MetaData()
Base = declarative_base()

users = Table(
    "users",
    metadata,
    Column("id", Integer, primary_key=True, index=True),
    Column("name", String, index=True),
    Column("age", Integer),
)

metadata.create_all(bind=engine)

# Templates setup
templates = Jinja2Templates(directory="templates")


@app.middleware("http")
async def db_session_middleware(request: Request, call_next):
    db_session = sessionmaker(autocommit=False, autoflush=False, bind=engine)
    request.state.db = db_session()
    response = await call_next(request)
    request.state.db.close()
    return response


@app.get("/", response_class=HTMLResponse)
async def read_upload_file(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})


@app.post("/uploadfile/")
async def create_upload_file(request: Request, file: UploadFile = Form(...), name_row: int = Form(...), age_row: int = Form(...)):
    content = await file.read()

    # Decode CSV content
    decoded_content = content.decode("utf-8").splitlines()

    # Parse CSV
    csv_reader = csv.reader(decoded_content)
    csv_data = list(csv_reader)

    # Extract name and age columns based on user selection
    name_column = [row[name_row] for row in csv_data]
    age_column = [row[age_row] for row in csv_data]

    # Save data to SQLite database
    with request.state.db.begin():
        for name, age in zip(name_column, age_column):
            request.state.db.execute(users.insert().values(name=name, age=age))

    return {"status": "Data saved successfully"}

In [ ]:
<!-- templates/index.html -->

<!DOCTYPE html>
<html>
<head>
    <title>CSV File Upload</title>
</head>
<body>
    <form action="/uploadfile/" method="post" enctype="multipart/form-data">
        <label for="file">Choose a CSV file:</label>
        <input type="file" name="file" accept=".csv" required>
        <br>
        <label for="name_row">Select Name column:</label>
        <input type="number" name="name_row" required>
        <br>
        <label for="age_row">Select Age column:</label>
        <input type="number" name="age_row" required>
        <br>
        <button type="submit">Submit</button>
    </form>
</body>
</html>
